In [2]:
import geopandas as gpd
from geopandas import *
import matplotlib.pyplot as plt
import pandas as pd

# Read in the Data Frame from Stata files and Clean the Dataframes

In [3]:
#IPUMS Wage data
Path_IPUMS = ("/Users/idiosyncrasy58/Dropbox/Documents/College/"+
              "Universitat Autonoma de Barcelona/IDEA - Economics/"+
              "Doctoral Thesis Ideas/Migration/IFLS/Project Files/"+
              "IPUMS/Project Files/Census Wage Data.dta")

#IFLS Wage and Schooling data by Province
Path_IFLS = ("/Users/idiosyncrasy58/Dropbox/Documents/College/"+
             "Universitat Autonoma de Barcelona/IDEA - Economics/"+
             "Doctoral Thesis Ideas/Migration/IFLS/Project Files/")

#Master Path
Master_path = ('/Users/idiosyncrasy58/Dropbox/Documents/College/'+
               'Universitat Autonoma de Barcelona/IDEA - Economics/'+
               'Programming/Coursera Courses/Python/Python for Everybody/'+
               'Course 5 - Capstone/Capstone Project/Visualizations/'+
               'Creation of Shape Files/' )

### IPUMS

In [4]:
#Read in the Data frame from Stata and clean: IPUMS Census Data

cols_keep = ["year", "r_wage_hr", "kabmov", "provmov", "MaxSchYrs", "perwt"]

Census_Data = ( pd.read_stata(Path_IPUMS, convert_categoricals=False)[cols_keep] 
                  .rename(columns={'provmov':'Prov Code','kabmov':'Kab Code'})
                  .reset_index()
                  .rename(columns={'index':'serial'}) )

#Drop if year is not 1995
Census_Data = Census_Data.loc[Census_Data["year"]!=1976]

#Recatogorize MaxSchYrs to 13 if it is greater than 12 years (at least some college)
Census_Data.ix[Census_Data["MaxSchYrs"]>12.0,"MaxSchYrs"]=13.0

#Change Kab Code to remove the province number that preceeds the Kab Code
Census_Data["Kab Code"] = ( (Census_Data["Kab Code"]/100)
                            .astype(str)
                            .str.extract('\.(\d*)',expand=False)
                            .astype('int8') )

In [5]:
Census_Data.head()

,serial,year,r_wage_hr,Kab Code,Prov Code,MaxSchYrs,perwt
31211,31211,1995,3.586087,13,54,6.0,30.0
31212,31212,1995,5.708466,13,54,12.0,30.0
31213,31213,1995,7.152790,13,54,12.0,30.0
31214,31214,1995,6.895257,13,54,6.0,30.0
31215,31215,1995,3.824791,13,54,6.0,30.0


### IFLS

#### Tracker File of individuals

In [6]:
cols_keep = ["wave", "pwt", "pidlink2", "provmov", "kabmov","MaxSchYrs", "flag_LastWave", "flag_OutSch", "flag_NotInSch"]
col_rename = {'wave':'year','pwt':'perwt','pidlink2':'serial','provmov':'Prov Code','kabmov':'Kab Code'}

Tracker = ( pd.read_stata(Path_IFLS+"MasterTrack2.dta", convert_categoricals=False)[cols_keep]
              .rename(columns=col_rename) )

#Keep only those people that have completed school or never went to school
Tracker = Tracker.loc[(Tracker["flag_LastWave"]==1) &
                      ((Tracker["flag_OutSch"]==1) | (Tracker["flag_NotInSch"]==1))]

#Drop the flag variables 
Tracker.drop(["flag_LastWave", "flag_OutSch", "flag_NotInSch"], axis=1, inplace=True)

#Replace several columns as an int and not as a double
Tracker[["serial","Prov Code","Kab Code"]] = Tracker[["serial","Prov Code","Kab Code"]].astype(int, copy=False)

In [7]:
Tracker.head()

,year,perwt,serial,Prov Code,Kab Code,MaxSchYrs
1,1997,1.388606,1060001,12,1,4.0
5,1993,1.218788,1060002,12,1,0.0
11,1997,2.172256,1060003,12,1,1.0
25,1993,0.931917,1060006,12,1,0.0
31,1997,1.222655,1080001,12,1,4.0


#### Wages of Individuals

In [8]:
cols_keep = ["year", "pidlink2", "r_wage_hr","job"]
col_rename = {'pidlink2':'serial'}

Wages = ( pd.read_stata(Path_IFLS+"Temp Files/Wage Database1.dta", convert_categoricals=False)[cols_keep]
            .rename(columns=col_rename) )

#Drop serial that has NaN
Wages = Wages[np.isfinite(Wages['serial'])]

#Replace serial as an int and not a double
Wages["serial"] = Wages["serial"].astype(int,copy=False)

#drop if job==2
Wages = Wages.loc[Wages["job"]!=2]
Wages.drop("job",axis=1, inplace=True)

##### Clean the Wages

First, make sure that we have only one yearly observation per person (so get the mean across year-persons)

In [9]:
Wages = Wages.groupby(["serial","year"],as_index=False)["r_wage_hr"].mean()

Now find those people whose last observation doesn't have a wage in it. We will fill it in with a collapse by the last observed wage and place it in the last observed year. Those who already have a wage in their last observed year keep that wage (the proceeding code is a way to create dummy variables based on logical conditions).

In [10]:
Wages['flag_LastObs'] = (Wages.serial != Wages.serial.shift(-1)).astype(int)
Wages['flag_miss_LastWage'] = ((Wages.flag_LastObs==1) & (np.isnan(Wages['r_wage_hr']))).astype(int)

Now we will keep in a new dataframe all those people who had a missing value in the position of where their last observation is and use the merge (a m:1 merge) as a trick to identify all their rows. We will then collapse after a groupby to have their last year observed filled in by a wage if they had a wage in the preceeding year(s).

In [11]:
Wages_Missing = Wages.ix[Wages["flag_miss_LastWage"]==1,["serial","flag_miss_LastWage"]]

#Drop flags from main dataframe
Wages.drop(["flag_miss_LastWage",'flag_LastObs'],axis=1,inplace=True)

#Find all rows of those who have missing wages in their last observed survey year
Wages_Missing = ( Wages.merge(Wages_Missing,how='inner',on="serial")
                       .groupby("serial", as_index=False)
                       .last()
                       .drop("flag_miss_LastWage",axis=1, inplace=False) )

In [12]:
#Replace the years of last observations to get only the wave years
Wages_Missing.ix[Wages_Missing["year"]==1992,"year"]=1993
Wages_Missing.ix[(Wages_Missing["year"]==1996)|(Wages_Missing["year"]==1995),"year"]=1997
Wages_Missing.ix[Wages_Missing["year"]==1999,"year"]=2000
Wages_Missing.ix[(Wages_Missing["year"]==2006)|(Wages_Missing["year"]==2008),"year"]=2007

#Drop NaN values in wages (no need to keep these people)
Wages_Missing.dropna(inplace=True)

Now we will update the Wages dataframe with the nonmissing values from the Wages_Missing dataframe. First we must set as the index the year and the serial number, otherwise the update process will be done on the index value (which, as we know, are just the integer values 0-len(dataframe)). So we will have some people's data update other's data based not on serial number, but rather on index value.

In [13]:
Wages.set_index(["serial","year"],inplace=True)
Wages_Missing.set_index(["serial","year"],inplace=True)

In [14]:
Wages.update(Wages_Missing)
del Wages_Missing

#### Merge Wages into Tracker File

In [15]:
Tracker = (Tracker.set_index(["serial","year"],inplace=False)
                  .merge(Wages, how='left',left_index=True,right_index=True,copy=False))

Read in the IFLS East wage data set to update values in Tracker data

In [16]:
cols_keep = cols_keep[0:3]

Wages_2012 = ( pd.read_stata(Path_IFLS+"Temp Files/2012 Wage Current.dta")[cols_keep]
                .rename(columns=col_rename))

Wages_2012[["serial","year"]] = Wages_2012[["serial","year"]].astype(int,copy=False)

Wages_2012.set_index(["serial","year"],inplace=True)

Update the Tracker data

In [17]:
Tracker.update(Wages_2012)
del Wages_2012

#### Append IPUMS File

Append the IPUMS file and then generate the log hourly wages (erasing the real wages)

In [18]:
Tracker = Tracker.reset_index().append(Census_Data,ignore_index=True)

In [19]:
Tracker["ln_wage_hr"]=np.log(Tracker["r_wage_hr"])
Tracker.drop("r_wage_hr",axis=1,inplace=True)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


Correct the provinces that split after 1993

In [20]:
Tracker.ix[Tracker["Prov Code"]==94,"Prov Code"]=91
Tracker.ix[Tracker["Prov Code"]==82,"Prov Code"]=81
Tracker.ix[Tracker["Prov Code"]==76,"Prov Code"]=73
Tracker.ix[Tracker["Prov Code"]==36,"Prov Code"]=32

Tracker = Tracker.loc[Tracker["Prov Code"]!=54]

# Generate the Provincial Median Wages and the Average Schooling Years

### Provincial Median Wages and Average Schooling

In [25]:
#Lambda function for weighted average of schooling years 
#wm = lambda x: np.average(x, weights=Tracker.loc[x.index,"perwt"])

#Define dictionary with functions to apply
function = {'MaxSchYrs':'mean','ln_wage_hr':'median'}

Avg_Prov = Tracker.groupby("Prov Code", as_index=False)[["MaxSchYrs","ln_wage_hr"]].agg(function)

In [26]:
Avg_Prov["r_wage_hr"]=np.exp(Avg_Prov["ln_wage_hr"])

# Import Shape Files

In [28]:
Ind_Prov = GeoDataFrame.from_file(Master_path+"Shape Files with Indonesian BPS Codes/Prov_merge/Prov_merge.shp")

### Merge in the statistical information into the Geopandas Dataframes

In [29]:
Ind_Prov = Ind_Prov.merge(Avg_Prov, how='left', left_on="Prov Code", right_on="Prov Code")

In [30]:
#Fix Missing Values based on the regrouping of Provinces to mimic the 1993
#IFLS indonesia (these provinces had data but were folded into the parent
#province from whence they came)

# Ind_Prov.ix[27,"r_wage_hr"]=Ind_Prov.ix[23,"r_wage_hr"]
# Ind_Prov.ix[27,"ln_wage_hr"]=Ind_Prov.ix[23,"ln_wage_hr"]
# Ind_Prov.ix[27,"Average Schooling"]=Ind_Prov.ix[23,"Average Schooling"]
# Ind_Prov.ix[27,"Median Schooling"]=Ind_Prov.ix[23,"Median Schooling"]

Ind_Prov.ix[19,"MaxSchYrs"]=5.550605

Ind_Prov.ix[15,"r_wage_hr"]=Ind_Prov.ix[11,"r_wage_hr"]
Ind_Prov.ix[15,"ln_wage_hr"]=Ind_Prov.ix[11,"ln_wage_hr"]
Ind_Prov.ix[15,"MaxSchYrs"]=Ind_Prov.ix[11,"MaxSchYrs"]

Ind_Prov.ix[28,"r_wage_hr"]=Ind_Prov.ix[25,"r_wage_hr"]
Ind_Prov.ix[28,"ln_wage_hr"]=Ind_Prov.ix[25,"ln_wage_hr"]
Ind_Prov.ix[28,"MaxSchYrs"]=Ind_Prov.ix[25,"MaxSchYrs"]

Ind_Prov.ix[30,"r_wage_hr"]=Ind_Prov.ix[29,"r_wage_hr"]
Ind_Prov.ix[30,"ln_wage_hr"]=Ind_Prov.ix[29,"ln_wage_hr"]
Ind_Prov.ix[30,"MaxSchYrs"]=Ind_Prov.ix[29,"MaxSchYrs"]

Ind_Prov.ix[32,"r_wage_hr"]=Ind_Prov.ix[31,"r_wage_hr"]
Ind_Prov.ix[32,"ln_wage_hr"]=Ind_Prov.ix[31,"ln_wage_hr"]
Ind_Prov.ix[32,"MaxSchYrs"]=Ind_Prov.ix[31,"MaxSchYrs"]

Ind_Prov = Ind_Prov.dropna()

In [31]:
Ind_Prov

,Prov Code,Province,geometry,MaxSchYrs,ln_wage_hr,r_wage_hr
0,11,Aceh,"(POLYGON ((95.1129443594893 5.566434697270201,...",9.478793,0.765713,2.150527
1,12,Sumatera Utara,(POLYGON ((98.56304499441775 1.610842788660926...,8.277768,0.535124,1.707661
2,13,Sumatera Barat,(POLYGON ((100.3340171959681 -3.28001314087794...,8.482733,0.645776,1.907467
3,14,Riau,(POLYGON ((102.4940984916833 1.129889926323819...,9.304348,0.938771,2.556838
4,15,Jambi,(POLYGON ((104.2475432565595 -1.03299575925923...,8.637455,0.739919,2.095765
5,16,Sumatera Selatan,(POLYGON ((104.567944798962 -1.751620662897045...,7.470047,0.429764,1.536895
6,17,Bengkulu,(POLYGON ((101.3849622989467 -2.34334692501442...,9.626186,0.773708,2.167790
7,18,Lampung,(POLYGON ((105.5067143072264 -5.92385168879810...,7.182378,0.247442,1.280745
8,19,Kep. Bangka Belitung,(POLYGON ((107.5251739146149 -3.19027266510658...,7.781690,0.752778,2.122890
9,21,Kep. Riau,(POLYGON ((103.4574064112914 0.640523894383146...,9.308511,0.762930,2.144552


# Plot the Chloropleths

### Wages

In [34]:
%matplotlib osx
( Ind_Prov.plot(column="r_wage_hr", cmap='Blues', scheme="quantiles", linewidth=0.1, legend=True)
         .axis('off') 
)

(90.0, 150.0, -12.0, 6.0)

In [ ]:
#plt.savefig('Prov_Wages.png', dpi=1000, frameon=False, bbox_inches='tight', facecolor='#F2F2F2')

### Schooling

In [36]:
%matplotlib osx
( Ind_Prov.plot(column="MaxSchYrs", cmap='Reds', scheme="quantiles", linewidth=0.1, legend=True)
         .axis('off') 
)

(90.0, 150.0, -12.0, 6.0)

In [ ]:
#plt.savefig('Prov_Education.png', dpi=1000, frameon=False, bbox_inches='tight', facecolor='#F2F2F2')